# 1. Data Gathering

### Using Reddit's API and PRAW

In [1]:
import json
import requests
import praw
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time
import re

In [2]:
# load API details from local .env file and set up praw object
load_dotenv()
reddit_read_only = praw.Reddit(client_id = os.getenv('client_id'),
                               client_secret = os.getenv('client_secret'),
                               user_agent = os.getenv('user_agent'))

In [3]:
df = pd.read_csv('./data/Whisky_Review_Archive.csv')
df.columns = ['timestamp', 'name', 'username', 'link', 'rating', 'region', 'price', 'date']
df.sample(3)

,timestamp,name,username,link,rating,region,price,date
4520,2/16/2013 14:25:25,Balvenie 12 Doublewood,TheRotundHobo,http://www.reddit.com/r/Scotch/comments/18na4c...,71,Speyside,27,2/16/2013
3544,11/10/2013 5:47:14,Auchentoshan 12,Hryelle,http://www.reddit.com/r/Scotch/comments/1p47cm...,86,Lowlands,NaN,10/24/2013
31192,3/15/2016 8:23:01,Miltonduff 20 1995 Signatory Un-Chillfiltered ...,LetThereBeR0ck,https://www.reddit.com/r/Scotch/comments/49g1w...,86,Speyside,NaN,03/07/16


In [4]:
#fails = []
#time_new = time.time()
#for i in range(0, 41355):
#    if i % 300 == 0:
#        time_old = time_new
#        time_new = time.time()
#        print('We\'ve done {} items.'.format(i), end=" ")
#        print('Time taken for last 300 is {:.1f} seconds'.format(time_new - time_old))
#    try:
#        url = df.iloc[i,3]
#        submission = reddit_read_only.submission(url=url).comments[0].body
#        df.loc[i,'review']=submission
#    except:
#        print('Failed at {}.'.format(i), end=" ")
#        fails.append(i)

# df.to_csv('./data/reviews_added.csv')
# Commented out as only needs running once - results stored in 'reviews_added.csv'

These results were stored in 'reviews_added.csv'. Getting reviews has failed for a few reasons here:

* /r/ScotchSwap was banned, so those reviews are unavailable. Those rows will simply have to be dropped.
* Some reviews are not the top comment, but are rather text submissions. I will need to get submission.selftext here for the review text.
* Annoyingly, I have lots of cases where I have comments that clearly aren't the review because the review is in the submission itself rather than the top comment. I'd like to avoid having to go through the entire list again to get the submission.selftext, so will look for 'reviews' that aren't reviews by looking at their length. It looks as though anything longer than 500 characters is definitely a review, so I can keep them. Then I'll just have to go through the rest, looks like.
* Some reviews are just missing. Those will have to be dropped.
* We have cases where someone has reviewed several whiskies at once. I should be able to sort this out once I start analysing the review texts. For instance, generally a review will cite tasting notes for 'nose', 'taste' or 'palette', 'finish', but not more than one. I should be able to identify expressions that isolate adjectives for each separate bit.

In [5]:
# Reopening pandas dataframe
df = pd.read_csv('./data/reviews_added.csv')
df.drop(columns = 'Unnamed: 0', inplace=True)

# Making copy to manipulate
df_copy = df.copy()
df_copy.sample(3)

,timestamp,name,username,link,rating,region,price,date,review
29002,4/7/2020 6:57:14,Linkwood 15 Gordon & MacPhail Distillery Label,TOModera,https://old.reddit.com/r/Scotch/comments/fwj8r...,62,Speyside,€70,04/07/20,The first time I had Linkwood it was chosen fo...
39334,NaN,W.L. Weller 12,FrunkLeftfoot,https://www.reddit.com/r/bourbon/comments/s1dh...,85,Bourbon,NaN,01/11/22,It was a good book. It could have been great. ...
1513,6/12/2015 15:47:58,AnCnoc Rutter,Whisky_Lads,http://www.reddit.com/r/Scotch/comments/29rzik...,84,Highlands,NaN,06/11/15,Here we go again..\n\n* **Nose:** Peaty new ma...


In [6]:
# Dropping nulls for the moment (I will try downloading submissions in a bit to see if that picks up any of these)
df_copy = df_copy[~df_copy['review'].isnull()]

# Calculating review length in characters 
df_copy['reviewlength'] = df_copy['review'].apply(lambda x: len(x))

# Selecting only those with at least 500 characters as those are likely to be actual reviews
df_copy = df_copy[df_copy['reviewlength']>500]
df_copy.shape

(36576, 10)

In [7]:
# These are the missing rows
missing = list(set(np.arange(0, 41355)) - set(df_copy.index.tolist()))
print('There are {} missing reviews out of {} potential reviews.'
      .format(len(missing), df.shape[0]))
# Going now to try to download submission.selftext for the missing ones to see if I can find 
# reviews there instead. 

#fails = []
#for i in missing:
#    try:
#        url = df.iloc[i,3]
#        submission = reddit_read_only.submission(url=url).selftext
#        df.loc[i,'review']=submission
#    except:
#        print('Failed at {}.'.format(i), end=" ")
#        fails.append(i)

# df.to_csv('submissions_added.csv')
# Commented out as only needs running once - results stored in 'submissions_added.csv'

There are 4779 missing reviews out of 41355 potential reviews.


In [8]:
# Reopening submissions_added pandas dataframe
df_sub = pd.read_csv('./data/submissions_added.csv')
df_sub.drop(columns = 'Unnamed: 0', inplace=True)

# Only need potential reviews I don't already have in df so slicing it to missing values only
df_sub = df_sub.loc[missing,:]

# removing nulls
df_sub = df_sub[~df_sub['review'].isnull()]

# Calculating review length in characters 
df_sub['reviewlength'] = df_sub['review'].apply(lambda x: len(x))

# Selecting only those with at least 100 characters as those will be actual reviews
df_sub = df_sub[df_sub['reviewlength']>100]
df_sub.shape

(1131, 10)

Some success (1131 potential reviews - stored in submissions_added.csv if you lose track of them), but I still have 3600 missing reviews, many of which are where I've obviously downloaded the wrong comment. What I'll do now is iterate through the same missing list, but this time I'll download the highest level comment that matches the author of the submission. This is probably what I should have done in the first place, but it should fix many of my remaining issues.

In [9]:
# Going now to try to download submission.selftext for the missing ones to see if I can find 
# reviews there instead.

#fails = []
#for i in missing:
#    try:
#        url = df.iloc[i,3]
#        submission = reddit_read_only.submission(url=url)
#        for comment in submission.comments:
#            if comment.author == submission.author:
#                df.loc[i,'review']=comment.body
#                break
#    except:
#        print('Failed at {}.'.format(i), end=" ")
#        fails.append(i)
#
# df.to_csv('./data/author_comments_added.csv')
# Commented out as only needs running once - results stored in 'author_comments_added.csv'

In [10]:
# Reopening submissions_added pandas dataframe
df_auth = pd.read_csv('./data/author_comments_added.csv')
df_auth.drop(columns = 'Unnamed: 0', inplace=True)

# Only need potential reviews I don't already have in df so slicing it to missing values only
df_auth = df_auth.loc[missing,:]

# removing nulls
df_auth = df_auth[~df_auth['review'].isnull()]

# Calculating review length in characters 
df_auth['reviewlength'] = df_auth['review'].apply(lambda x: len(x))

# Selecting only those with at least 100 characters as those will be actual reviews
df_auth = df_auth[df_auth['reviewlength']>100]
df_auth.shape

(3900, 10)

Dataframes available at the moment:
* df - reviews_added.csv
* df_copy - reviews_added_.csv but with 4779 length<500 character 'reviews' removed. These were all generated by just using submission.comment\[0\].body. 
* df_sub - 1131 potential reviews from submission.selftext.
* df_auth - 3900 potential reviews from getting the first top-level comment that matches the author of the submission.

Will now investigate the conflicts between the 1131 and the 3900.

In [11]:
conflict_list = sorted(set(df_auth.index.tolist()) & set(df_sub.index.tolist()))
sample_ten = np.random.choice(conflict_list, 10)
for i in sample_ten:
    print('\n\n\n\nAuthor Comment is\n\n\n\n')
    print(df_auth.loc[i,'review'])
    print('\n\n\n\nSubmission is\n\n\n\n')
    print(df_sub.loc[i,'review'])





Author Comment is




**Talisker 10** 
 
* **ABV:** 45.8% 
* **Nose** Sea breeze, brine, peat, salt. It smells like I'm on the beach. 
* **Palate:** A peaty-er version of Balvanie malt... but only just. Hints of peat shine through. Banana? An off-putting note, like wet bread. 
* **Finish:** Seawee, vanilla, and... Salty! 
* **Score:** 68/100 
 
**Overall:** This one just doesn't do it for me. The salt really overwhelms everything else, and the palate is pretty one dimensional. 

 
**Talisker Dark Storm**   
   
* **ABV:** 45.8% 
* **Nose:** Cracked black pepper, salt. 
* **Palate:** Pepper, vanilla, and, you guessed it, salt! Wet, soggy bread. 
* **Finish:** Salted caramels, honey. Much sweeter than the 10 year. Fades into an oak spice. 
* **Score:** 72/100 

**Overall:** While this is a step up from the 10 year, it only adds a little dimension through the pepper notes. That off-putting wet bread flavor is still present in the palate, though not quite as prominent. Overall the salt

It looks as though the two methods are generating the same output a lot of the time, which is reassuring. But let's just check:

In [12]:
for i in conflict_list:
    if df_auth.loc[i,'review'] == df_sub.loc[i,'review']:
        pass
    else:
        print('\n\n\n\nAuthor Comment is\n\n\n\n')
        print(df_auth.loc[i,'review'])
        print('\n\n\n\nSubmission is\n\n\n\n')
        print(df_sub.loc[i,'review'])





Author Comment is




You're not going to lose anything other than delaying the day you open it and taste a good scotch, or sell it. With how many good scotches are out there, You're probably not going to make a million dollars on it in the next ten years. 

Unless you move in the next several years and you drop the box its in. Or you have a teenage son who finds it and drinks it and replaces it with juice, for you to discover when you try and sell it. Then you lose it all.

I say, drink it, and enjoy it





Submission is




I'm very new to scotch, so I don't know much about these sorts of things. I know that scotch doesn't continue to age like wine. However, considering JWG has been discontinued, I thought it would be fun to buy a bottle and hold onto it. Do you think its value will increase in the years to come? 

NOTE: I COULD drink it, but I want to also buy a single malt to enjoy. I plan on drinking Laphroaig 10 and holding onto the JWG.




Author Comment is




It's intere

The good news here is that it looks like the submissions.selftext method is always better at getting the review over the author comment method, so I can simply add all the reviews from df_sub and then add any leftover from df_auth. That should leave me with a relatively decent dataset that I can then start to analyse to extract tasting notes.

In [13]:
df_new = pd.concat([df_copy, df_sub], axis=0)
df_new = pd.concat([df_new, df_auth[~df_auth.index.isin(df_new.index.tolist())]], axis=0)

df_new.to_csv('./data/gathering_complete.csv')

That's it for data gathering. Lots of work still to be done in terms of extracting information from the reviews, but I will do that in a different Notebook file.